In [1]:
# Add PyDatAnalysis to path
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, "/Users/owensheekey/Documents/Research/PyDatAnalysis")

export_path = 'Exports/'

In [41]:
from __future__ import annotations
from progressbar import progressbar
from src.DatObject.Make_Dat import get_dat, get_dats
import src.UsefulFunctions as U
from src.DataStandardize.ExpSpecific.Feb21 import Feb21Exp2HDF, Feb21ExpConfig
from src.DataStandardize.ExpConfig import ExpConfigGroupDatAttribute, ExpConfigBase
import multiprocessing as mp
import plotly.graph_objs as go
import numpy as np
import lmfit as lm
from typing import TYPE_CHECKING, Iterable, Optional
from src.DatObject.Attributes.Transition import i_sense_digamma, i_sense, i_sense_digamma_quad
from src.UsefulFunctions import edit_params
from src.DatObject.Attributes.SquareEntropy import square_wave_time_array, integrate_entropy
import logging
logger = logging.getLogger(__name__)
import src.UsefulFunctions as U
import scipy.io
from scipy.interpolate import RectBivariateSpline

In [3]:
def narrow_fit(dat, width, **kwargs):
    '''
    Get a fit only including +/- width in dat.x around center of transition
    kwargs is the stuff to pass to get_fit
    Return a fit
    '''
    out = dat.SquareEntropy.get_Outputs(existing_only=True)
    x = np.copy(out.x)
    y = np.copy(out.averaged)
    y = np.mean(y[(0, 2), :], axis=0)

    start_ind = np.nanargmin(np.abs(np.add(x, width)))
    end_ind = np.nanargmin(np.abs(np.subtract(x, width)))

    x[:start_ind] = [np.nan] * start_ind
    x[end_ind:] = [np.nan] * (len(x) - end_ind)

    y[:start_ind] = [np.nan] * start_ind
    y[end_ind:] = [np.nan] * (len(y) - end_ind)

    fit = dat.SquareEntropy.get_fit(
        x=x,
        data=y,
        **kwargs)
    return fit

In [4]:
def narrow_fit_trans_only(dat, width, **kwargs):
    '''
    Get a fit only including +/- width in dat.x around center of transition
    kwargs is the stuff to pass to get_fit
    Return a fit
    '''
    x = np.copy(dat.Transition.avg_x)
    y = np.copy(dat.Transition.avg_data)

    start_ind = np.nanargmin(np.abs(np.add(x, width)))
    end_ind = np.nanargmin(np.abs(np.subtract(x, width)))

    x[:start_ind] = [np.nan] * start_ind
    x[end_ind:] = [np.nan] * (len(x) - end_ind)

    y[:start_ind] = [np.nan] * start_ind
    y[end_ind:] = [np.nan] * (len(y) - end_ind)

    fit = dat.SquareEntropy.get_fit(
        x=x,
        data=y,
        **kwargs)
    return fit

In [112]:
def do_calc(datnum, overwrite=True):
    """Just a function which can be passed to a process pool for faster calculation"""
    save_name = 'SPS.01'

    dat = get_dat(datnum)

    setpoints = [0.01, None]

    # Get other inputs
    setpoint_times = square_wave_time_array(dat.SquareEntropy.square_awg)
    sp_start, sp_fin = [U.get_data_index(setpoint_times, sp) for sp in setpoints]
    logger.debug(f'Setpoint times: {setpoints}, Setpoint indexs: {sp_start, sp_fin}')

    # Run Fits
    pp = dat.SquareEntropy.get_ProcessParams(name=None,  # Load default and modify from there
                                             setpoint_start=sp_start, setpoint_fin=sp_fin,
                                             transition_fit_func=i_sense,
                                             save_name=save_name)
    out = dat.SquareEntropy.get_Outputs(name=save_name, inputs=None, process_params=pp, overwrite=overwrite)
    dat.Entropy.get_fit(which='avg', name=save_name, data=out.average_entropy_signal, x=out.x, check_exists=False,
                        overwrite=overwrite)
    [dat.Entropy.get_fit(which='row', row=i, name=save_name,
                         data=row, x=out.x, check_exists=False,
                         overwrite=overwrite) for i, row in enumerate(out.entropy_signal)]
    return out

In [6]:
def get_deltaT(dat):
    """Returns deltaT of a given dat in mV"""
    ho1 = dat.AWG.max(0)  # 'HO1/10M' gives nA * 10
    t = dat.Logs.temps.mc

    # Datnums to search through (only thing that should be changed)
    datnums = set(range(1312, 1451+1)) - set(range(1312, 1451+1, 4))
    # datnums = set()
    # for j in range(5):
    #     datnums = datnums.union(set(range(28 * j + 1312, 28 * j + 1312 + 4 * 7 + 1)) - set([28 * j + 1312 + 4 * i for i in range(8)]))
    # datnums = list(datnums)

    dats = get_dats(datnums)

    dats = [d for d in dats if np.isclose(d.Logs.temps.mc, dat.Logs.temps.mc, rtol=0.1)]  # Get all dats where MC temp is within 10%
    bias_lookup = np.array([d.Logs.fds['HO1/10M'] for d in dats])

    indp = np.argmin(abs(bias_lookup - ho1))
    indm = np.argmin(abs(bias_lookup + ho1))
    theta_z = np.nanmean([d.Transition.avg_fit.best_values.theta for d in dats if d.Logs.fds['HO1/10M'] == 0])

    # temp_lookup = np.array([d.Logs.temps.mc for d in dats])
    # bias_lookup = np.array([d.Logs.fds['HO1/10M'] for d in dats])
    #
    # indp = np.argmin(temp_lookup - t + bias_lookup - ho1)
    # indm = np.argmin(temp_lookup - t + bias_lookup + ho1)
    # indz = np.argmin(temp_lookup - t + bias_lookup)

    theta_p = dats[indp].Transition.avg_fit.best_values.theta
    theta_m = dats[indm].Transition.avg_fit.best_values.theta
    # theta_z = dats[indz].Transition.avg_fit.best_values.theta
    return (theta_p + theta_m) / 2 - theta_z

In [240]:
NRG = scipy.io.loadmat('Results.mat')
occ = NRG["Occupation_mat"]
ens = np.reshape(NRG["Ens"], 401)
ts  = np.reshape(NRG["Ts"], 70)
ens = np.flip(ens)
occ = np.flip(occ, 0)
interp = RectBivariateSpline(ens, np.log10(ts), occ, kx=1,ky=1)
def interpNRG(x, dx, ampconst, amplin, center, lin, const, logt):
    ens = np.multiply(np.add(x, center),dx)
    occ = [1 - interp(en, logt)[0][0] for en in ens]
    lin_amp = np.multiply(ens, amplin)
    amp = np.add(lin_amp, ampconst)
    result = occ * amp + const + lin*x
    return result

In [113]:
# datnums = set(range(1869, 1919)) - set(range(1870, 1919, 2))
# transdatnums = set(range(1869, 1919)) - set(range(1869, 1919, 2))
datnums = np.sort(list(set(range(2089, 2095)) - set(range(2090, 2095, 2))))
transdatnums = np.sort(list(set(range(2089, 2095)) - set(range(2089, 2095, 2))))

In [114]:
dats = get_dats(list(datnums), overwrite=False)
transdats = get_dats(list(transdatnums), overwrite=False)

In [115]:
escs = [dat.Logs.fds["ESC"] for dat in dats]
datnums = [dat.datnum for dat in dats]

In [116]:
out = [do_calc(dn) for dn in progressbar(datnums)]

100% (3 of 3) |##########################| Elapsed Time: 0:07:44 Time:  0:07:44


In [27]:
Θ = 3.9
fit = transdats[0].Transition.get_fit(which='avg', check_exists=False)
params = fit.params
params.add('g', value=0, vary=True, min=-50, max=1000)
new_pars = edit_params(params, param_name='theta', value=Θ, vary=False) 

In [28]:
amp_digamma_ = [narrow_fit_trans_only(
    dat,
    600,
    which='avg', 
    initial_params=new_pars, 
    fit_func=i_sense_digamma, 
    check_exists=False).best_values.amp
for dat in progressbar(transdats)]

100% (3 of 3) |##########################| Elapsed Time: 0:01:30 Time:  0:01:30


In [45]:
amp_digamma_

[0.8915637252820501, 0.6737286118654073, 0.48944639212080143]

In [29]:
g_digamma_ = [narrow_fit_trans_only(
    dat,
    600,
    initial_params=new_pars, 
    fit_func=i_sense_digamma, 
    check_exists=False).best_values.g
for dat in progressbar(transdats)]

100% (3 of 3) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [30]:
amp_digamma_cold = [narrow_fit(
    dat,
    600,
    which='avg', 
    initial_params=new_pars, 
    fit_func=i_sense_digamma, 
    check_exists=False).best_values.amp
for dat in progressbar(dats)]

100% (3 of 3) |##########################| Elapsed Time: 0:00:01 Time:  0:00:01


In [31]:
g_digamma_cold = [narrow_fit(
    dat,
    600,
    initial_params=new_pars, 
    fit_func=i_sense_digamma, 
    check_exists=False).best_values.g
for dat in progressbar(dats)]

100% (3 of 3) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [32]:
mids_digamma_ = [narrow_fit(
    dat,
    600,
    initial_params=new_pars, 
    fit_func=i_sense_digamma, 
    check_exists=False).best_values.mid
for dat in progressbar(dats)]

100% (3 of 3) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [241]:
nrg_pars = lm.Parameters()
nrg_pars.add_many(
    ('dx', -0.0007, True, None, None, None, None),
    ('ampconst', 2, True, 0.1, None, None, None),
    ('amplin', 0.01, True, None, None, None, None),
    ('center', 0, True, None, None, None, None),
    ('lin', 0.1, True, 0.001, None, None, None),
    ('const', 0, True, None, None, None, None),
    ('logt', -2, False, None, None, None, None))

In [242]:
fit0 = narrow_fit_trans_only(
    transdats[0],
    1000,
    initial_params=nrg_pars, 
    fit_func=interpNRG, 
    check_exists=False, overwrite=True)

In [243]:
print(fit0.fit_report)

[[Model]]
    Model(interpNRG)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 57
    # data points      = 1505
    # variables        = 6
    chi-square         = 0.00101987
    reduced chi-square = 6.8037e-07
    Akaike info crit   = -21365.9606
    Bayesian info crit = -21334.0613
[[Variables]]
    dx:       -0.00271179 +/- 1.8832e-06 (0.07%) (init = -0.0007)
    ampconst:  0.90199648 +/- 9.7128e-05 (0.01%) (init = 2)
    amplin:    0.00309079 +/- 1.9652e-04 (6.36%) (init = 0.01)
    center:   -2.16684036 +/- 0.00333531 (0.15%) (init = 0)
    lin:       0.00148281 +/- 3.6295e-07 (0.02%) (init = 0.1)
    const:     6.78861248 +/- 6.5543e-05 (0.00%) (init = 0)
    logt:     -2 (fixed)
[[Correlations]] (unreported correlations are < 0.100)
    C(lin, const)       = -0.885
    C(ampconst, const)  = -0.705
    C(amplin, lin)      =  0.665
    C(ampconst, lin)    =  0.618
    C(amplin, const)    = -0.579
    C(dx, center)       = -0.434
    C(dx, ampconst)    

In [247]:
nrg_pars = lm.Parameters()
nrg_pars.add_many(
    ('dx', -0.0007, True, None, None, None, None),
    ('ampconst', 2, True, 0.1, None, None, None),
    ('amplin', 0.01, True, -0.01, 0.01, None, None),
    ('center', 0, True, None, None, None, None),
    ('lin', 0.1, True, 0.001, None, None, None),
    ('const', 0, True, None, None, None, None),
    ('logt', -4, False, None, None, None, None))

In [248]:
fit1 = narrow_fit_trans_only(
    transdats[1],
    500,
    initial_params=nrg_pars, 
    fit_func=interpNRG, 
    check_exists=False, overwrite=True)

In [249]:
print(fit1.fit_report)

[[Model]]
    Model(interpNRG)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 14000
    # data points      = 845
    # variables        = 6
    chi-square         = 49.9044747
    reduced chi-square = 0.05948090
    Akaike info crit   = -2378.69593
    Bayesian info crit = -2350.25991
##  Warning: uncertainties could not be estimated:
[[Variables]]
    dx:        0.00799352 (init = -0.0007)
    ampconst:  0.10498756 (init = 2)
    amplin:   -9.9989e-05 (init = 0.01)
    center:    317.268088 (init = 0)
    lin:       0.00100050 (init = 0.1)
    const:     7.52098729 (init = 0)
    logt:     -4 (fixed)


In [227]:
nrg_pars = lm.Parameters()
nrg_pars.add_many(
    ('dx', -0.0007, True, None, 0, None, None),
    ('ampconst', 2, True, 0.1, None, None, None),
    ('amplin', 0.01, True, None, None, None, None),
    ('center', 0, True, None, None, None, None),
    ('lin', 0.1, True, 0.001, None, None, None),
    ('const', 0, True, None, None, None, None),
    ('logt', -5, False, None, None, None, None))

In [228]:
fit2 = narrow_fit_trans_only(
    transdats[2],
    700,
    initial_params=nrg_pars, 
    fit_func=interpNRG, 
    check_exists=False, overwrite=True)

In [229]:
fit2.best_values

dx=-8.8218e-05
ampconst=294.62
amplin=0.001
center=7781.9
lin=0.001
const=6.1519
logt=-5

In [230]:
nrgfits = [fit0, fit1, fit2]

In [37]:
fig = go.Figure()

fig.add_trace(go.Scatter(mode='markers', x=escs, y=amp_digamma_cold, text=datnums, name="Entropy fits"))
fig.add_trace(go.Scatter(mode='markers', x=escs, y=amp_digamma_, text=transdatnums, name="Transition fits"))

fig.update_layout(xaxis_title='ESC/ mV', yaxis_title='Amplitude /nA',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="browser")

In [58]:
fig.write_html(f'Exports/Ampls_03_12_dats{datnums[0]}_{datnums[-1]}.html')

In [59]:
fig = go.Figure()

fig.add_trace(go.Scatter(mode='markers', x=escs, y=np.array(g_digamma_cold)/Θ, text=datnums, name="Entropy fits"))
fig.add_trace(go.Scatter(mode='markers', x=escs, y=np.array(g_digamma_)/Θ, text=transdatnums, name="Transition fits"))
fig.update_layout(xaxis_title='ESC/ mV', yaxis_title='Gamma/Theta /arb',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')

fig.show(renderer="browser")

In [60]:
fig.write_html(f'Exports/Gammas_03_12_dats{datnums[0]}_{datnums[-1]}.html')

In [214]:
fig = go.Figure()
for i, dat in progressbar(enumerate(transdats)):
    x = np.copy(dat.Transition.avg_x)
    y = np.copy(dat.Transition.avg_data)
    
    xfit = np.linspace(-2000,2000,1001)

    yfit = nrgfits[i].eval_fit(xfit) - nrgfits[i].best_values.lin*xfit
    fig.add_trace(go.Scatter(mode='markers', x=x, y=y - nrgfits[i].best_values.lin*x, name=f'{transdatnums[i]}d'))
    fig.add_trace(go.Scatter(mode='lines', x=xfit, y=yfit, name=f'{transdatnums[i]}f', marker_color='grey'))
fig.update_layout(xaxis_title='ACC/100 /mV', yaxis_title='Current /nA',
                      title=f'Dat {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="browser")

| |#                                                  | 2 Elapsed Time: 0:00:00


In [235]:
dat = get_dat(2089)
data = dat.SquareEntropy.get_Outputs(name='SPS.01').average_entropy_signal
ii = dat.Entropy.get_integration_info()
dx = ii.dx
dt = ii.dT
x = dat.SquareEntropy.avg_x
i = 0
ens = np.multiply(np.add(x, nrgfits[i].best_values.center), nrgfits[i].best_values.dx)
val = [1 - interp(en, nrgfits[i].best_values.logt)[0][0] for en in ens]

amp = nrgfits[i].best_values.ampconst + np.multiply(nrgfits[i].best_values.amplin,val)
integrated = np.nancumsum(data, axis=-1) * dx / amp / dt
fig = go.Figure()
fig.add_trace(go.Scatter(mode='markers', x=x, y=amp))
fig.show(renderer="browser")

In [118]:
fig.write_html(f'Exports/NRG_fits_linear_amp{datnums[0]}_{datnums[-1]}.html')

In [38]:
deltaT = [get_deltaT(dat) for dat in dats]
ampl = amp_digamma_cold
for i, dat in enumerate(dats):
    dat.Entropy.set_integration_info(dT=deltaT[i], amp=ampl[i], overwrite=True)

In [39]:
fig = go.Figure()
int_ents = []
for i in range(len(dats)):
    int_ent = integrate_entropy(out[i].average_entropy_signal, dats[i].Entropy.integration_info.sf)
    int_ents.append(int_ent[-1])
    fig.add_trace(go.Scatter(mode='markers', 
                             x=np.subtract(out[i].x, mids_digamma_[i]), 
                             y=int_ent,
                             name= f'g/t:{np.divide(g_digamma_,Θ)[i]:.2f}, dat{dats[i].datnum}, ESS:{dats[i].Logs.fds["ESS"]}'))

fig.add_trace(go.Scatter(mode='lines', x=[-400,400], y=[np.log(2), np.log(2)], name="Log2"))  
fig.add_trace(go.Scatter(mode='lines', x=[-400,400], y=[np.log(3), np.log(3)], name="Log3"))  
fig.update_layout(xaxis_title='ACC/100 /mV', yaxis_title='Entropy /kb',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer='browser')

In [41]:
fig.write_html(f'Exports/IntEntropy_03_12_dats{datnums[0]}_{datnums[-1]}.html')

In [363]:
# escs1 = []
# escs2 = []
# int_ents1 = []
# int_ents2 = []
# datnums1 = []
# datnums2 = []
# for i in range(len(escs)):
#     if dats[i].datnum >= 1941:
#         escs2.append(escs[i])
#         int_ents2.append(int_ents[i])
#         datnums2.append(dats[i].datnum)
#     else:
#         escs1.append(escs[i])
#         int_ents1.append(int_ents[i])
#         datnums1.append(dats[i].datnum)

In [79]:
ents = [dat.Entropy.get_fit(which='avg', name="SPS.005", check_exists=True).best_values.dS for dat in dats]
fig = go.Figure()
fig.add_trace(go.Scatter(mode='markers+lines', 
                         x=escs, y=int_ents, 
                         text=[dat.datnum for dat in dats], 
                         name="Int"))
fig.add_trace(go.Scatter(mode='markers+lines', 
                         x=escs, y=ents, 
                         text=[dat.datnum for dat in dats], 
                         name="Fit"))
# fig.add_trace(go.Scatter(mode='markers+lines', 
#                          x=escs1, y=int_ents1, 
#                          text=datnums1, 
#                          name="Set 1 -- 10 nA"))
# fig.add_trace(go.Scatter(mode='markers+lines', 
#                          x=escs2, y=int_ents2, 
#                          text=datnums2, 
#                          name="Set 2 -- 10 nA"))
# #                          marker=dict(color='LightSkyBlue',
#                                      size=10,
#                                      line=dict(color='DarkSlateGrey',
#                                                width=2)),
#                          error_y=dict(type='data', # value of error bar given in data coordinates
#                                       array=[0.05]*len(int_ents),
#                                       visible=True)))
# fig.add_trace(go.Scatter(mode='lines', x=[-500,0], y=[np.log(2), np.log(2)], name="Log2", line=dict(color='DarkSlateGrey', width=4, dash='dot')))  

fig.update_layout(xaxis_title='ESC /mV', yaxis_title='Entropy /kb',
                      title=f'Integrated -- Dats {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="browser")

In [45]:
fig.write_html(f'Exports/IntEntropyvalues_03_12_dats{datnums[0]}_{datnums[-1]}.html')

In [36]:
dats[2].Logs.fds["ESC"]

-189.21